In [1]:
%run ./utils/commonImports.py
%run ./utils/tradingImports.py
%matplotlib inline

# Load Data

In [2]:
def load_and_divide(db, table, from_date=None, to_date=None, train_ratio=0.5, period=None):
    data = load_trading_data(db, table, from_date, to_date, period)
    data_train, data_test = divide_train_and_test(data, train_ratio)
    return data, data_train, data_test


db = 'D:\\Dropbox\\My work\\krypl-project\\sqlite\\ploniex-chart-data\\USDT_BTC.db'
data, data_train, data_test = load_and_divide(db, 'chart_data', from_date='2017-01-01', period='5min')
ohlc, ohlc_train, ohlc_test = data.copy(), data_train.copy(), data_test.copy()

# Calculation

In [48]:
import time

maxs = []
till_maxs = []
mins = []
df = ohlc.copy()
DAY = 1440
TARGET = 0.01

def _max_row(day_df, col):
    i = day_df[col].idxmax()
    max_row = day_df.iloc[i, :]
    return max_row

def _min_row(day_df, col):
    i = day_df[col].idxmin()
    min_row = day_df.iloc[i, :]
    return min_row

def _max(day_df):
    close_max_row = _max_row(day_df, 'close')
    open_max_row = _max_row(day_df, 'open')
    return (close_max_row, 'close') if close_max_row['close'] > open_max_row['open'] else (open_max_row, 'open')

def _fullfiled_row(row, day_df, col, target):
    day_df['R'] = _return(row['close'], day_df[col])
    fullfiled = day_df.query(f"R >= {target}")
    if fullfiled.shape[0] > 0:
        return fullfiled.iloc[0, :]
    else:
        return day_df.iloc[-1, :]

def fullfiled(row, day_df, target):
    close_row = _fullfiled_row(row, day_df, 'close', target)
    open_row = _fullfiled_row(row, day_df, 'open', target)
    return (close_row, 'close') if close_row['close'] > open_row['open'] else (open_row, 'open')

def _return(v1, v2):
    return (v2 / v1) - 1


start_time = time.time()
for i in range(df.shape[0] - 1):
    if i % 1000 == 0:
        print(i, '%.3f s' % (time.time() - start_time))
        start_time = time.time()
    
    day_i = i + DAY if i + DAY < df.shape[0] else df.shape[0]-1
    row = df.iloc[i, :]
    day_df = df.iloc[i+1:day_i, :].reset_index().drop('index', axis=1).reset_index()

    
    fullfiled_row, fullfiled_col = fullfiled(row, day_df, TARGET)
    
    max_index = fullfiled_row['index'] if fullfiled_row['index'] > 0 else 1
    day_df = day_df.iloc[:max_index, :]
    min_row = _min_row(day_df, 'low')
    
    maxs.append(_return(row['close'], fullfiled_row[fullfiled_col]))
    till_maxs.append(fullfiled_row['index'])
    mins.append(_return(row['close'], min_row['low']))

0 0.000 s
1000 11.076 s
2000 10.689 s
3000 10.700 s
4000 10.787 s
5000 11.336 s
6000 10.898 s
7000 10.657 s
8000 10.768 s
9000 10.902 s
10000 10.896 s
11000 10.866 s
12000 10.699 s
13000 10.900 s
14000 10.863 s
15000 10.936 s
16000 10.846 s
17000 10.936 s
18000 10.968 s
19000 10.677 s
20000 10.794 s
21000 10.750 s
22000 10.574 s
23000 10.893 s
24000 10.934 s
25000 11.224 s
26000 10.963 s
27000 10.977 s
28000 10.957 s
29000 10.774 s
30000 10.716 s
31000 10.873 s
32000 10.868 s
33000 10.914 s
34000 10.831 s
35000 11.018 s
36000 10.787 s
37000 10.899 s
38000 10.726 s
39000 10.832 s
40000 10.921 s
41000 10.943 s
42000 10.681 s
43000 10.988 s
44000 10.930 s
45000 11.562 s
46000 10.802 s
47000 10.662 s
48000 10.796 s
49000 10.949 s
50000 10.702 s
51000 10.688 s
52000 10.707 s
53000 10.949 s
54000 10.623 s
55000 10.597 s
56000 10.590 s
57000 10.887 s
58000 10.861 s
59000 12.331 s
60000 11.019 s
61000 10.884 s
62000 10.855 s
63000 10.927 s
64000 10.867 s
65000 10.947 s
66000 10.645 s
67000 10.

IndexError: single positional indexer is out-of-bounds

In [49]:
maxs += [-1, -1]
till_maxs += [-1, -1]
mins += [-1, -1]

In [50]:
df = ohlc.copy()
df['maxReturn'] = maxs
df['tillMax'] = till_maxs
df['minReturn'] = mins

In [51]:
df

,timestamp,period,open,high,low,close,volume,quoteVolume,weightedAverage,date,maxReturn,tillMax,minReturn
228868,1483225200,5min,964.0000,964.0000,964.0000,964.0000,3.7374,0.0039,964.0000,2017-01-01 00:00:00,0.0109,142,-0.0072
228869,1483225500,5min,962.8600,962.8600,962.8600,962.8600,74.4955,0.0774,962.8600,2017-01-01 00:05:00,0.0121,141,-0.0061
228870,1483225800,5min,964.0000,964.0000,962.8600,962.8600,273.3070,0.2837,963.2768,2017-01-01 00:10:00,0.0121,140,-0.0061
228871,1483226100,5min,964.0000,964.0000,964.0000,964.0000,69.1332,0.0717,964.0000,2017-01-01 00:15:00,0.0109,139,-0.0072
228872,1483226400,5min,964.0000,965.0000,962.8600,965.0000,"3,050.4950",3.1637,964.2074,2017-01-01 00:20:00,0.0106,139,-0.0083
228873,1483226700,5min,963.8600,965.0000,962.8600,965.0000,215.8639,0.2242,962.9156,2017-01-01 00:25:00,0.0106,138,-0.0083
228875,1483227000,5min,965.0000,965.0000,965.0000,965.0000,0.0000,0.0000,965.0000,2017-01-01 00:30:00,0.0106,137,-0.0083
228876,1483227300,5min,964.0000,965.0000,963.0000,963.0463,36.1246,0.0375,963.9131,2017-01-01 00:35:00,0.0119,135,-0.0063
228877,1483227600,5min,964.0633,965.0050,964.0633,965.0050,549.9110,0.5699,964.8749,2017-01-01 00:40:00,0.0106,135,-0.0083
228878,1483227900,5min,965.0050,965.0050,965.0050,965.0050,0.0000,0.0000,965.0050,2017-01-01 00:45:00,0.0106,134,-0.0083


In [52]:
write_tsv(df, 'D:\\Dropbox\\My work\\krypl-project\\analysis\\return-01-within-day.tsv')

# Analysis

In [58]:
df = read_tsv('D:\\Dropbox\\My work\\krypl-project\\analysis\\return-01-within-day.tsv')
df = df[['date', 'maxReturn', 'tillMax', 'minReturn']].head(df.shape[0] - 2)

In [59]:
df.shape

(160953, 4)

In [60]:
df.shape

(160953, 4)

In [61]:
df.query('maxReturn >= 0.01')

,date,maxReturn,tillMax,minReturn
0,2017-01-01 00:00:00,0.0109,142,-0.0072
1,2017-01-01 00:05:00,0.0121,141,-0.0061
2,2017-01-01 00:10:00,0.0121,140,-0.0061
3,2017-01-01 00:15:00,0.0109,139,-0.0072
4,2017-01-01 00:20:00,0.0106,139,-0.0083
5,2017-01-01 00:25:00,0.0106,138,-0.0083
6,2017-01-01 00:30:00,0.0106,137,-0.0083
7,2017-01-01 00:35:00,0.0119,135,-0.0063
8,2017-01-01 00:40:00,0.0106,135,-0.0083
9,2017-01-01 00:45:00,0.0106,134,-0.0083


In [ ]:
df

In [62]:
df.query('maxReturn >= 0.01').shape[0] / df.shape[0]

0.9025057004218623

In [63]:
query = '(maxReturn > 0.01)'
df_true = df.query(query)
df_false = df.query(f'not {query}')

In [64]:
df_true.describe()

,maxReturn,tillMax,minReturn
count,"145,261.0000","145,261.0000","145,261.0000"
mean,0.0132,149.6656,-0.0194
std,0.0043,244.0073,0.0303
min,0.0100,0.0000,-0.3172
25%,0.0108,15.0000,-0.0220
50%,0.0119,49.0000,-0.0089
75%,0.0139,158.0000,-0.0031
max,0.0930,"1,438.0000",0.0067


In [65]:
df_false.describe()

,maxReturn,tillMax,minReturn
count,"15,692.0000","15,692.0000","15,692.0000"
mean,-0.0958,"1,404.5342",-0.1663
std,0.0677,154.5982,0.0838
min,-0.4036,0.0000,-0.4272
25%,-0.1277,"1,438.0000",-0.2125
50%,-0.0858,"1,438.0000",-0.1479
75%,-0.0468,"1,438.0000",-0.1082
max,0.0096,"1,438.0000",-0.0001
